In [6]:
import os
os.environ["ZHIPUAI_API_KEY"] = '056744e429f9da058272391d1107ebe6.7aXbE4ZlIuYn7yoL'

import asyncio
import platform
from typing import Any

import fire

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.team import Team


class SpeakAloud(Action):
    """Action: Speak out aloud in a debate (quarrel)"""

    PROMPT_TEMPLATE: str = """
    ## BACKGROUND
    Suppose you are {name}, you are in a debate with {opponent_name}.
    ## DEBATE HISTORY
    Previous rounds:
    {context}
    ## YOUR TURN
    Now it's your turn, you should closely respond to your opponent's latest argument, state your position, defend your arguments, and attack your opponent's arguments,
    craft a strong and emotional response in 80 words, in {name}'s rhetoric and viewpoints, your will argue:
    """
    name: str = "SpeakAloud"

    async def run(self, context: str, name: str, opponent_name: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context, name=name, opponent_name=opponent_name)
        # logger.info(prompt)

        rsp = await self._aask(prompt)

        return rsp


class Debator(Role):
    name: str = ""
    profile: str = ""
    opponent_name: str = ""

    def __init__(self, **data: Any):
        super().__init__(**data)
        self._init_actions([SpeakAloud])
        self._watch([UserRequirement, SpeakAloud])

    async def _observe(self) -> int:
        await super()._observe()
        # accept messages sent (from opponent) to self, disregard own messages from the last round
        self.rc.news = [msg for msg in self.rc.news if msg.send_to == {self.name}]
        return len(self.rc.news)

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # An instance of SpeakAloud

        memories = self.get_memories()
        context = "\n".join(f"{msg.sent_from}: {msg.content}" for msg in memories)
        # print(context)

        rsp = await todo.run(context=context, name=self.name, opponent_name=self.opponent_name)

        msg = Message(
            content=rsp,
            role=self.profile,
            cause_by=type(todo),
            sent_from=self.name,
            send_to=self.opponent_name,
        )
        self.rc.memory.add(msg)

        return msg


async def debate(idea: str, investment: float = 3.0, n_round: int = 5):
    """Run a team of presidents and watch they quarrel. :)"""
    Biden = Debator(name="Biden", profile="Democrat", opponent_name="Trump")
    Trump = Debator(name="Trump", profile="Republican", opponent_name="Biden")
    team = Team()
    team.hire([Biden, Trump])
    team.invest(investment)
    team.run_project(idea, send_to="Biden")  # send debate topic to Biden and let him speak first
    await team.run(n_round=n_round)


async def main(idea: str, investment: float = 3.0, n_round: int = 10):
    """
    :param idea: Debate topic, such as "Topic: The U.S. should commit more in climate change fighting"
                 or "Trump: Climate change is a hoax"
    :param investment: contribute a certain dollar amount to watch the debate
    :param n_round: maximum rounds of the debate
    :return:
    """
    if platform.system() == "Windows":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    await debate(idea, investment, n_round)


if __name__ == "__main__":
    await main(idea = "Trump: Climate change is a hoax")

/root/miniconda3/lib/python3.10/site-packages/pydantic/_internal/_core_utils.py:210: RuntimeWarning: coroutine 'debate' was never awaited
  schema = self._schema_type_to_method[schema['type']](schema.copy(), f)
2024-05-19 22:16:57.442 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-19 22:16:57.516 | INFO     | metagpt.config:get_default_llm_provider_enum:126 - API: LLMProviderEnum.ZHIPUAI
2024-05-19 22:16:57.518 | INFO     | metagpt.team:invest:86 - Investment: $3.0.
2024-05-19 22:16:57.523 | INFO     | __main__:_act:58 - Biden(Democrat): to do SpeakAloud(SpeakAloud)


 Mr. President, your denial of climate change is deeply troubling. The science is clear: it's not a hoax, but a real and urgent threat to our planet and future generations. Our economy, national security, and very way of life are at stake. Your rejection of facts and science only weakens our position in the global fight against climate change. Together, we must take decisive action to reduce emissions, invest in clean energy, and protect our beautiful lands and waters. Americans deserve a leader who faces the facts and acts with courage, not one who panders to fossil fuel interests.

2024-05-19 22:17:05.381 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 121, completion_tokens: 121
2024-05-19 22:17:05.385 | INFO     | __main__:_act:58 - Trump(Republican): to do SpeakAloud(SpeakAloud)


 Mr. Biden, you’re wrong on climate change. The economy, not the environment, is my top priority. Your liberal policies will kill jobs and hurt American workers. I’ve made energy independence a reality, reducing dependence on foreign oil and creating jobs in the process. My administration has invested in clean energy, and we’re leading the way in innovation. Your attacks on fossil fuels are attacks on American workers, and I won’t stand for it. We need to protect our economy and way of life, not apologize for being energy leaders. Together, we can achieve energy dominance and keep America

2024-05-19 22:17:12.381 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 237, completion_tokens: 124
2024-05-19 22:17:12.386 | INFO     | __main__:_act:58 - Biden(Democrat): to do SpeakAloud(SpeakAloud)


...

 Mr. President, your stubborn denial of climate change and defense of fossil fuels is a betrayal of our planet and future generations. The green New Deal is a moral imperative, not a socialist fantasy, to save our planet and secure a prosperous future for all. We can create millions of good-paying jobs and grow our economy by investing in clean energy and cutting-edge technologies. Your claim to energy independence is built on the backs of exploited workers and a dangerous reliance on fossil fuels. It's time to embrace a new era of clean energy innovation and prioritize the well-being of all Americans. Join me in taking real action on climate change and creating an economy that works for everyone. Together, we can tackle this global challenge and ensure a healthy, prosperous future for generations to come

2024-05-19 22:18:28.908 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 1637, completion_tokens: 162
2024-05-19 22:18:28.914 | INFO     | __main__:_act:58 - Trump(Republican): to do SpeakAloud(SpeakAloud)


. Mr. Biden, you’re relentlessly pushing a socialist agenda that would bankrupt our country and destroy millions of jobs. Your green New Deal is a fantasy that panders to environmental extremists, and I won’t let it happen. I’ve delivered energy independence, jobs, and a strong economy while investing in clean energy. We must prioritize American workers and our economy, not apologize for our success.

Climate change is complex, and your naive approach only weakens our position in the global fight against it. I’ve stood up to Greta Thunberg and others like her, and I won’t back down. We must protect our economy, secure a bright future for our children, and continue leading the way in innovation and energy dominance.

Together, we’ll create jobs, grow the economy, and keep America great – without pandering to the socialist agenda. Americans deserve a leader who puts them first, not one who sells out to the radical left. Reject the green New Deal and join me in fighting for a prosperous, 

2024-05-19 22:18:41.598 | INFO     | metagpt.utils.cost_manager:update_cost:48 - Total running cost: $0.001 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 1794, completion_tokens: 225


.